In [29]:
import requests
from bs4 import BeautifulSoup
myBaseUrl = "https://www.amazon.com/SafeStraw-Drinking-Aid-Dysphagia-Fluids/product-reviews/B00XWEOU4I/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber="
start_urls = []
for i in range(1, 5): # there are four page numbers to cycle through
    start_urls.append(myBaseUrl + str(i))
HEADERS = {
    'authority': 'www.amazon.com',
'method' : 'POST',
'path' : '/gp/redirection/singapore.html',
'scheme' : 'https',
'accept':'*/*',
'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
    
}
response = []
for i in start_urls:
    response.append(requests.get(i,headers=HEADERS))


In [21]:
print(response) #response 200 means success

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [36]:
#collects soup data
soups = []
for i in response:
    soups.append(BeautifulSoup(i.content))
    
#collects and cleans review data
reviews = []
for soup in soups:
    review = soup.findAll("span",{'class':"a-size-base review-text review-text-content"})
    for i in review:
        reviews.append(i.text)
strippedreviews = []
for review in reviews:
    strippedreviews.append(review.strip())

In [24]:
soups[0]

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a

In [63]:
avg_rating = 3.9 #change this average as necessary
stars = []
for soup in soups:
    star = soup.findAll("span",{'class':"a-icon-alt"})
    for i in star: 
        stars.append(i.text)
strippedstars = [] #this list still contains the stars from the avg. rating, top positive review, and top negative review
for star in stars:
    strippedstars.append(star.strip())
#code to remove excess stars scraped from html 
finalstars = []
indexofavg_rating = 0; 
for i in range(len(strippedstars)):
    if str(avg_rating) in strippedstars[i]:
        indexofavg_rating = i
    elif i != indexofavg_rating:
        if i != indexofavg_rating + 1:
            if i != indexofavg_rating + 2:
                finalstars.append(strippedstars[i])

35


In [64]:
#collects and cleans title data 
titles = []
for soup in soups:
    title = soup.findAll("a",{'class':"a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    for i in title:
        titles.append(i.text)
strippedtitles = []
for title in titles:
    strippedtitles.append(title.strip())


In [70]:
#creates a csv file with title and review data 
import pandas as pd    

df = pd.DataFrame(strippedtitles, columns=["title"])   
df.to_csv('soupsafestrawthin.csv', index=False)  #change name of csv file as needed 
read = pd.read_csv("soupsafestrawthin.csv")
read["review"] = strippedreviews
read.to_csv("soupsafestrawthin.csv", index=False)
read["star"] = finalstars
read.to_csv("soupsafestrawthin.csv", index=False)
df = pd.read_csv("soupsafestrawthin.csv")

In [71]:
#runs sentiment analysis on collected data in csv file
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity


df['polarity'] = df['review'].apply(pol)
df['subjectivity'] = df['review'].apply(sub)
df

,title,review,star,polarity,subjectivity
0,Get rid of thickened liquids!!,The SafeStaw is almost a miracle. After a stro...,5.0 out of 5 stars,0.353333,0.577115
1,Nurses at our hospital came to see how this wo...,My wife is swalling problems and we needed to ...,5.0 out of 5 stars,0.293750,0.368750
2,DO YOU WANT TO CHOKE?,"BOUGH THIS DEVICE FOR DYSPHAGIA, BUT MAKES ME ...",1.0 out of 5 stars,0.250000,0.288462
3,Worked great for a couple of weeks for my daug...,Worked great for a couple of weeks for my daug...,5.0 out of 5 stars,0.623214,0.660714
4,Provides small sip to help with swallowing and...,Great product! Follow directions and out works,5.0 out of 5 stars,1.000000,0.750000
5,Great invention!,Great idea!,5.0 out of 5 stars,1.000000,0.750000
6,Doesn’t work properly,Does not work. Can’t suck out if it.,1.0 out of 5 stars,0.000000,0.000000
7,Good product,Used this to help my granny with her newly dis...,5.0 out of 5 stars,0.545455,0.601515
8,worked perfectly for my mother who would not u...,worked perfectly for my mother who would not u...,5.0 out of 5 stars,0.348148,0.496296
9,Five Stars,perfect,5.0 out of 5 stars,1.000000,1.000000
